In [1]:
import cv2
import dlib
import numpy as np
import time

In [2]:
# Load the PNG image of the clown nose
clown_nose = cv2.imread("clown_nose.png", cv2.IMREAD_UNCHANGED)
blush = cv2.imread("blush.png", cv2.IMREAD_UNCHANGED)
glasses = cv2.imread('glasses.png', cv2.IMREAD_UNCHANGED)

# Initialize video capture and face detector
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

start_time = time.time()
total_latency = 0
num_frames = 0

In [3]:
# Variables to toggle features
show_landmarks = False
show_clown_nose = False
show_blush = False
show_glasses = False

In [4]:
def toggle_filter(event, x, y, flags, param):
    global show_landmarks, show_clown_nose, show_blush, show_glasses
    
    # Check if left mouse button is clicked
    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if the click is within the region of the "Landmarks" button
        if 10 <= x <= 110 and 10 <= y <= 40:
            show_landmarks = not show_landmarks
        # Check if the click is within the region of the "Clown Nose" button
        elif 130 <= x <= 250 and 10 <= y <= 40:
            show_clown_nose = not show_clown_nose
        # Check if the click is within the region of the "Blush" button
        elif 270 <= x <= 360 and 10 <= y <= 40:
            show_blush = not show_blush
        # Check if the click is within the region of the "Glasses" button
        elif 380 <= x <= 470 and 10 <= y <= 40:
            show_glasses = not show_glasses

In [5]:
def draw_landmarks(frame, landmarks):
    # Draw circles at each facial landmark point
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        cv2.circle(frame, (x, y), 3, (255, 0, 0), -1)

In [6]:
def apply_clown_nose(frame, landmarks):
    # Extract the coordinates of the nose tip (landmark 33)
    nose_tip = (landmarks.part(33).x, landmarks.part(33).y)
    
    # Resize the clown nose to fit the size of the nose
    width = int(landmarks.part(16).x - landmarks.part(0).x) // 3
    height = int(clown_nose.shape[0] * (width / clown_nose.shape[1]))
    resized_nose = cv2.resize(clown_nose, (width, height))
    
    # Calculate the position to place the clown nose
    x_offset = nose_tip[0] - width // 2
    y_offset = nose_tip[1] - height // 2 - 10  # Adjust height up
    
    # Create a region of interest (ROI) in the frame
    roi = frame[y_offset:y_offset + height, x_offset:x_offset + width]
    
    # Add the clown nose image to the ROI using alpha blending
    alpha_s = resized_nose[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    
    for c in range(0, 3):
        roi[:, :, c] = (alpha_s * resized_nose[:, :, c] + alpha_l * roi[:, :, c])
    
    # Place the modified ROI back into the frame
    frame[y_offset:y_offset + height, x_offset:x_offset + width] = roi

In [7]:
def apply_blush(frame, landmarks):
    # Calculate the size and position for the blush overlay
    width = int((landmarks.part(16).x - landmarks.part(0).x) / 3)
    height = int(blush.shape[0] * (width / blush.shape[1]))
    resized_blush = cv2.resize(blush, (width, height))
    
    # Calculate the positions for the blush on both cheeks
    left_cheek = (landmarks.part(3).x - width // 2, landmarks.part(3).y - height // 1)
    right_cheek = (landmarks.part(13).x - width // 2, landmarks.part(13).y - height // 1)
    
    # Function to apply the blush on a cheek
    def overlay_blush(cheek_position):
        x, y = cheek_position
        # Create a region of interest (ROI)
        roi = frame[y:y + height, x:x + width]
        
        # Add the blush image to the ROI using alpha blending
        alpha_s = resized_blush[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        
        for c in range(0, 3):
            roi[:, :, c] = (alpha_s * resized_blush[:, :, c] + alpha_l * roi[:, :, c])
        
        # Place the modified ROI back into the frame
        frame[y:y + height, x:x + width] = roi
    
    # Apply the blush to both cheeks
    overlay_blush(left_cheek)
    overlay_blush(right_cheek)

In [8]:
def apply_glasses(frame, landmarks):
    
    left_eye_x, left_eye_y = landmarks.part(36).x, landmarks.part(36).y
    right_eye_x, right_eye_y = landmarks.part(45).x, landmarks.part(45).y
    
    width = int(abs(right_eye_x - left_eye_x) * 1.5)
    height = int(width * glasses.shape[0] / glasses.shape[1])
    
    glassesResized = cv2.resize(glasses, (width, height))
    
    glasses_x = int((left_eye_x + right_eye_x) / 2 - width / 2)
    glasses_y = int((left_eye_y + right_eye_y) / 2 - height / 2)
    
    for y in range(glassesResized.shape[0]):
        for x in range(glassesResized.shape[1]):
            if glassesResized[y, x, 3] != 0:
                frame[glasses_y + y, glasses_x + x] = glassesResized[y, x, 0:3]

In [9]:
# Create a resizable window
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Frame", 800, 600)

# Register mouse callback function
cv2.setMouseCallback("Frame", toggle_filter)


In [10]:
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = detector(gray)
    
    # Draw interactive buttons
    cv2.rectangle(frame, (10, 10), (110, 40), (0, 0, 255),)
    cv2.putText(frame, "Landmarks", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.rectangle(frame, (130, 10), (250, 40), (0, 0, 255),)
    cv2.putText(frame, "Clown Nose", (145, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.rectangle(frame, (270, 10), (360, 40), (0, 0, 255),)
    cv2.putText(frame, "Blush", (295, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.rectangle(frame, (380, 10), (470, 40), (0, 0, 255),)
    cv2.putText(frame, "Glasses", (390, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    for face in faces:
        # Get facial landmarks
        landmarks = predictor(gray, face)
        
        # Toggle features based on flags
        if show_landmarks:
            draw_landmarks(frame, landmarks)
        
        if show_clown_nose:
            apply_clown_nose(frame, landmarks)
            
        if show_blush:
            apply_blush(frame, landmarks)
            
        if show_glasses:
            apply_glasses(frame, landmarks)
    
    # Display the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    
    current_time = time.time()
    latency = current_time - start_time
    start_time = current_time
    
    total_latency += latency
    num_frames += 1
    
    # Handle keyboard input to toggle features and exit
    if key == ord('l'):  # Press 'l' to toggle landmarks
        show_landmarks = not show_landmarks
    elif key == ord('c'):  # Press 'c' to toggle clown nose
        show_clown_nose = not show_clown_nose
    elif key == ord('b'):  # Press 'b' to toggle blush filter
        show_blush = not show_blush
    elif key == ord('g'): # Press 'g' to toggle glasses filter
        show_glasses = not show_glasses
    elif key == 27:  # Press 'Esc' to exit
        break
    

if num_frames > 0:
    average_latency = total_latency / num_frames
    print("Latency: ", average_latency)
    
# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()



Latency:  0.048296906731345436
